# Micrograd
We're now going to codify our observations from backpropagation and will define an autograd-engine that will automatically calculating gradients based on the inputs

## Backward based on operations
- Every operation defines it's backward process based on the chain rule - ``_backward()`` function
- The ``_backward()`` function by default does nothing, and this is true for all leaf noode (nodes after which no operation is perfomed)
- For all other operations the ``_backward()`` function is defined as a closure, and then assigned to the attribute of ``_backward()`` of that object


The ``_backward()`` function can now be called in the correct order on the node and fills in the gradients of the graph based on the operations.

## Backward for one node
To avoid having to manually determine the order in which the ``_backward()`` functions have to be called, we will use topological sorting to automatically determine the order and then call the function based on the order.

$\Rightarrow$ This enabled us to only having to call ``backward()`` on the individual node and it will recursivelly iterate throught the graph and call the intermediate ``_backward()``s


In [ ]:
import math

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op: str = None, label: str = ""):
        self.data = data
        self.grad = 0.0
        self._prev = set(_children)
        self._op = _op
        self.label = label
        # This will perform the chain rule based on the operation
        self._backward: callable = lambda: None

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        out = Value(self.data + other.data, _children=(self, other), _op="+")

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out

    def __mul__(self, other):
        out = Value(self.data * other.data, _children=(self, other), _op="*")

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, _children=(self,), _op="tanh")

        def _backward():
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):  # Needs to be reversed because we start at the end
            node._backward()

In [ ]:
# Visualizing the computation graph
from graphviz import Digraph


def trace(root):
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges


# for any value use a rectangle, for any operation use a circle
def draw_graph(value: Value):
    dot = Digraph(format="svg", graph_attr={"rankdir": "LR"})  # Left to right
    nodes, edges = trace(value)
    # For each node, add a rectangle with the value
    for n in nodes:
        uid = str(id(n))
        dot.node(
            name=uid,
            label="{%s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad),
            shape="record",
        )
        # For any operation, use a circle
        if n._op:
            dot.node(name=uid + n._op, label=n._op)
            # Add edges to the graph
            dot.edge(uid + n._op, uid)
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    return dot

# Example

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label="x1")
x2 = Value(0.0, label="x2")
# weights wl,w2
w1 = Value(-3.0, label="w1")
w2 = Value(1.0, label="w2")
# bias of the neuron
b = Value(6.8813735870195432, label="b")  # Value set so the numbers come out "nice"
# x1*w1 + x2*w2 + b
x1w1 = x1 * w1
x1w1.label = "x1*w1"
x2w2 = x2 * w2
x2w2.label = "x2*w2"
x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = "x1*w1 + x2*w2"
n = x1w1x2w2 + b
n.label = "n"
o = n.tanh()
o.label = "o"

# Calling ``_backward()`` manually
We're now going to let the ``_backward()`` function of the operations compute the gradients instead of doing it manually.

For this we have to call the function in the right order beginning at the end

In [ ]:
draw_graph(o)

The base case is always $1$, and since ``other.grad = 0`` not setting the base case will result in an error. Therefore our gradient at the output is $1$

In [ ]:
o.grad = 1.0

In [ ]:
o._backward()

In [ ]:
n._backward()

In [ ]:
b._backward()

In [ ]:
x1w1x2w2._backward()

In [ ]:
x1w1._backward()

In [ ]:
x2w2._backward()

# Automatically computing the full graph
We now would like to call everything in one go. We never want to call the ``_backward()`` functions individually.

To perfom a backward pass of a node we have to ensure that we call tha backward on all of it's children first, because those gradients influence the current node.

-> **We have to compute everything that needs to computed before computing the current gradient!**

For this the graph needs to be in a layout where all the edges flow into one direction so that the gradients can be perfomed accordingly. This can be achieved by applying **topological sort** on the graph which returns such a layout!

<img src="../img/topological-sort.webp" alt="Topological Sort Example">

This example show the original DAG (Directed acyclic graph) and it's two possible topological orderings

**How it works:**
- We start at a root node (in our case the last node in the graph ``o``)
- We mark the node as visited
- We then recurively visit all it's children first and make them add themselves to the ``topo`` list
- **We only add the current nodes after all it's children have been added**
  - This guarentees us that all the nodes flow into one direction, and that out children will be processed first!

In [ ]:
topo = []
visited = set()


def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)

In [ ]:
# Starting at the root node (output node)
build_topo(o)

In [ ]:
topo

In [ ]:
o.grad = 1.0

for node in reversed(topo):  # Needs to be reversed because we start at the end
    node._backward()

In [ ]:
o.backward()

In [ ]:
draw_graph(o)

# Using variables more than once

If we're using a variable more than once such as
```python
a = Value(3.0, label='a')
b = a + a; b.label='b'
b.backward()
draw_graph(b)
```
we're getting:

<img src='../img/multiple-use-variable-wrong-result.svg' alt='Wrong result when using a variable multiple time'>

**This result is wrong** as the gradient of $a$ has to be $2$, because it is used twice and the $+$ node distributes the gradients to it's children, so it should distribute $1$ twice because of the double usage.

The reason it's not doing this is because when a variable is used more than once, what happens during the backward pass is that the ``_backward()`` is called twice, and because were setting the ``self.grad`` in the ``_backward()`` function of the operation, we override the old gradient, leading to an incorrect answer.

Old implementation that reproduces the bug:
```python
def _backward():
    self.grad = 1.0 * out.grad
    other.grad = 1.0 * out.grad
```

Instead of multiple usage of a variable (which is the general idea), the gradients accumulate, meaning they add up over time. Therefore we just need to add a ``+`` in front of the equals to ensure accumulation:
```python
def _backward():
    self.grad += 1.0 * out.grad
    other.grad += 1.0 * out.grad
```

This now works correctly because of the implementation above

In [ ]:
a = Value(3.0, label="a")
b = a + a
b.label = "b"
b.backward()
draw_graph(b)